__[pycuda learning](https://github.com/apowers313/roc/blob/master/experiments/2024.08.30-07.27.46-pycuda-learning/2024.08.30-07.27.46-pycuda-learning.ipynb)__

In [1]:
!date

Sat Aug 31 15:35:38 PDT 2024


In [1]:
import os

os.environ["PATH"] = os.environ["PATH"] + ":/usr/local/cuda/bin"
print("PATH:", os.environ["PATH"])

PATH: /usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/snap/bin:/usr/local/cuda/bin


In [4]:
!nvidia-smi

Mon Sep  2 12:12:12 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070 ...    Off |   00000000:01:00.0 Off |                  N/A |
|  0%   50C    P8              4W /  220W |       2MiB /  12282MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [6]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Mon_Oct_24_19:12:58_PDT_2022
Cuda compilation tools, release 12.0, V12.0.76
Build cuda_12.0.r12.0/compiler.31968024_0


In [1]:
# https://github.com/PacktPublishing/Hands-On-GPU-Programming-with-Python-and-CUDA/blob/master/Chapter03/deviceQuery.py

import pycuda.driver as drv

drv.init()

print(f"Detected {drv.Device.count()} CUDA Capable device(s)")

for i in range(drv.Device.count()):
    gpu_device = drv.Device(i)
    print(f"Device {gpu_device.name()}:")
    compute_capability = float("%d.%d" % gpu_device.compute_capability())
    print(f"\tCompute Capability: {compute_capability}")
    print(f"\tTotal Memory: {gpu_device.total_memory()//(1024**2)}")

    raw_device_attributes = gpu_device.get_attributes()
    device_attributes = {str(k): raw_device_attributes[k] for k in raw_device_attributes.keys()}

    num_mp = device_attributes["MULTIPROCESSOR_COUNT"]

    major_compute_capability = gpu_device.compute_capability()[0]
    cuda_cores_per_mp = {
        # Maxwell
        5.0: 128,
        5.1: 128,
        5.2: 128,
        # Pascal
        6.0: 64,
        6.1: 128,
        6.2: 128,
        # Volta and Turing
        7.0: 64,
        7.5: 64,
        # Ampere
        8.0: 64,
        8.6: 128,
        8.9: 128,  # Ada Lovelace
        # Hopper
        9.0: 128,
    }[major_compute_capability]

    # RTX 4070 SUPER should have 7168 CUDA Cores, this reports 3584 which isn't right
    # https://www.nvidia.com/en-us/geforce/graphics-cards/40-series/rtx-4070-family/
    print(
        f"\t({num_mp}) Multiprocessors, ({cuda_cores_per_mp}) CUDA Cores / Multiprocessor: {num_mp*cuda_cores_per_mp} CUDA Cores"
    )

    device_attributes.pop("MULTIPROCESSOR_COUNT")

    for k in device_attributes.keys():
        print(f"\t{k}: {device_attributes[k]}")

Detected 1 CUDA Capable device(s)
Device NVIDIA GeForce RTX 4070 SUPER:
	Compute Capability: 8.9
	Total Memory: 12002
	(56) Multiprocessors, (64) CUDA Cores / Multiprocessor: 3584 CUDA Cores
	ASYNC_ENGINE_COUNT: 2
	CAN_MAP_HOST_MEMORY: 1
	CAN_USE_HOST_POINTER_FOR_REGISTERED_MEM: 1
	CLOCK_RATE: 2475000
	COMPUTE_CAPABILITY_MAJOR: 8
	COMPUTE_CAPABILITY_MINOR: 9
	COMPUTE_MODE: DEFAULT
	COMPUTE_PREEMPTION_SUPPORTED: 1
	CONCURRENT_KERNELS: 1
	CONCURRENT_MANAGED_ACCESS: 1
	DIRECT_MANAGED_MEM_ACCESS_FROM_HOST: 0
	ECC_ENABLED: 0
	GENERIC_COMPRESSION_SUPPORTED: 1
	GLOBAL_L1_CACHE_SUPPORTED: 1
	GLOBAL_MEMORY_BUS_WIDTH: 192
	GPU_OVERLAP: 1
	HANDLE_TYPE_POSIX_FILE_DESCRIPTOR_SUPPORTED: 1
	HANDLE_TYPE_WIN32_HANDLE_SUPPORTED: 0
	HANDLE_TYPE_WIN32_KMT_HANDLE_SUPPORTED: 0
	HOST_NATIVE_ATOMIC_SUPPORTED: 0
	INTEGRATED: 0
	KERNEL_EXEC_TIMEOUT: 0
	L2_CACHE_SIZE: 50331648
	LOCAL_L1_CACHE_SUPPORTED: 1
	MANAGED_MEMORY: 1
	MAXIMUM_SURFACE1D_LAYERED_LAYERS: 2048
	MAXIMUM_SURFACE1D_LAYERED_WIDTH: 32768
	MAXIMUM_

In [3]:
import numpy as np
import pycuda.autoinit
from pycuda import gpuarray

host_data = np.array([1, 2, 3, 4, 5], dtype=np.float32)
device_data = gpuarray.to_gpu(host_data)
device_data_x2 = 2 * device_data
host_data_x2 = device_data_x2.get()
print(host_data_x2)

[ 2.  4.  6.  8. 10.]


In [5]:
import pycuda.autoinit
import pycuda.driver as drv
from pycuda import gpuarray
from pycuda.compiler import SourceModule

ker = SourceModule(
    """
__global__ void hello_world_ker()
{
	printf("Hello world from thread %d, in block %d!\\n", threadIdx.x, blockIdx.x);
	
	__syncthreads();
	
	if(threadIdx.x == 0 && blockIdx.x == 0)
	{
		printf("-------------------------------------\\n");
		printf("This kernel was launched over a grid consisting of %d blocks,\\n", gridDim.x);
		printf("where each block has %d threads.\\n", blockDim.x);
	}
}
"""
)

hello_ker = ker.get_function("hello_world_ker")
hello_ker(block=(5, 1, 1), grid=(2, 1, 1))

Hello world from thread 0, in block 1!
Hello world from thread 1, in block 1!
Hello world from thread 2, in block 1!
Hello world from thread 3, in block 1!
Hello world from thread 4, in block 1!
Hello world from thread 0, in block 0!
Hello world from thread 1, in block 0!
Hello world from thread 2, in block 0!
Hello world from thread 3, in block 0!
Hello world from thread 4, in block 0!
-------------------------------------
This kernel was launched over a grid consisting of 2 blocks,
where each block has 5 threads.


In [6]:
# https://github.com/PacktPublishing/Hands-On-GPU-Programming-with-Python-and-CUDA/blob/master/Chapter04/simple_scalar_multiply_kernel.py
import pycuda.autoinit
import pycuda.driver as drv
import numpy as np
from pycuda import gpuarray
from pycuda.compiler import SourceModule

ker = SourceModule(
    """
__global__ void scalar_multiply_kernel(float *outvec, float scalar, float *vec)
{
     int i = threadIdx.x;
     outvec[i] = scalar*vec[i];
}
"""
)

scalar_multiply_gpu = ker.get_function("scalar_multiply_kernel")

testvec = np.random.randn(512).astype(np.float32)
testvec_gpu = gpuarray.to_gpu(testvec)
outvec_gpu = gpuarray.empty_like(testvec_gpu)

scalar_multiply_gpu(outvec_gpu, np.float32(2), testvec_gpu, block=(512, 1, 1), grid=(1, 1, 1))

print(f"Does our kernel work correctly? : {np.allclose(outvec_gpu.get() , 2*testvec)}")

Does our kernel work correctly? : True


In [7]:
import pycuda.autoinit
from pycuda.compiler import SourceModule

mod = SourceModule(
    """
__global__ void test_kernel()
{
    #define MY_X (threadIdx.x + blockIdx.x * blockDim.x)
    #define MY_Y (threadIdx.y + blockIdx.y * blockDim.y)
    #define MY_Z (threadIdx.z + blockIdx.z * blockDim.z)

    //printf("gridDim %d %d %d, warp %d\\n", gridDim.x, gridDim.y, gridDim.z, warpSize);
    //printf("Hello world from thread %d, in block %d!\\n", threadIdx.x, blockIdx.x);
    printf("(%d, %d, %d): Block (%d, %d, %d), Thread (%d, %d, %d)\\n", MY_X, MY_Y, MY_Z, blockIdx.x, blockIdx.y, blockIdx.z, threadIdx.x, threadIdx.y, threadIdx.z);
}
"""
)

test_kernel = mod.get_function("test_kernel")
test_kernel(block=(4, 2, 2), grid=(2, 1, 1))

(4, 0, 0): Block (1, 0, 0), Thread (0, 0, 0)
(5, 0, 0): Block (1, 0, 0), Thread (1, 0, 0)
(6, 0, 0): Block (1, 0, 0), Thread (2, 0, 0)
(7, 0, 0): Block (1, 0, 0), Thread (3, 0, 0)
(4, 1, 0): Block (1, 0, 0), Thread (0, 1, 0)
(5, 1, 0): Block (1, 0, 0), Thread (1, 1, 0)
(6, 1, 0): Block (1, 0, 0), Thread (2, 1, 0)
(7, 1, 0): Block (1, 0, 0), Thread (3, 1, 0)
(4, 0, 1): Block (1, 0, 0), Thread (0, 0, 1)
(5, 0, 1): Block (1, 0, 0), Thread (1, 0, 1)
(6, 0, 1): Block (1, 0, 0), Thread (2, 0, 1)
(7, 0, 1): Block (1, 0, 0), Thread (3, 0, 1)
(4, 1, 1): Block (1, 0, 0), Thread (0, 1, 1)
(5, 1, 1): Block (1, 0, 0), Thread (1, 1, 1)
(6, 1, 1): Block (1, 0, 0), Thread (2, 1, 1)
(7, 1, 1): Block (1, 0, 0), Thread (3, 1, 1)
(0, 0, 0): Block (0, 0, 0), Thread (0, 0, 0)
(1, 0, 0): Block (0, 0, 0), Thread (1, 0, 0)
(2, 0, 0): Block (0, 0, 0), Thread (2, 0, 0)
(3, 0, 0): Block (0, 0, 0), Thread (3, 0, 0)
(0, 1, 0): Block (0, 0, 0), Thread (0, 1, 0)
(1, 1, 0): Block (0, 0, 0), Thread (1, 1, 0)
(2, 1, 0):

# Jupyter
At this point I could make a PyCUDA cell magic :)

https://ipython.readthedocs.io/en/stable/config/custommagics.html

In [8]:
from IPython.core.magic import register_line_magic, register_cell_magic, register_line_cell_magic


@register_cell_magic
def cmagic(line, cell):
    "my cell magic"
    return line, cell

In [9]:
%%cmagic foo

this is a test

foo bar

('foo', '\nthis is a test\n\nfoo bar\n')

In [10]:
@register_cell_magic
def cuda(line, cell):
    "Runs NVIDIA CUDA C code via PyCUDA"
    notebook_path = os.path.abspath("")
    mod = SourceModule(cell)
    run_me = mod.get_function("run_me")
    run_me(block=(4, 1, 1))

In [11]:
%%cuda
__global__ void run_me()
{
    printf("hello world\n");
}

hello world
hello world
hello world
hello world


In [10]:
# %%cuda test_kernel arg1 arg2 block=(1,2,3) grid=(4,5,6)
# requires parsing ^

no syntax highlighting in magic cells :(

maybe a custom CUDA Kernel?

https://jupyter-client.readthedocs.io/en/latest/kernels.html

https://ipython.readthedocs.io/en/stable/install/kernel_install.html

# CudaSourceFile
Instead of going the Jupyter route, I'll just use external files for all the IDE
goodness. Also enables reusability like headerfiles.

In [2]:
import os
import pycuda.autoinit
from pycuda.compiler import SourceModule

notebook_path = os.path.abspath("")


class CudaSourceFile:
    def __init__(
        self, filename: str, kernels: list[str] = list(), include_dirs: list[str] = list()
    ) -> None:
        with open(filename) as f:
            file_str = f.read()
            self.mod = SourceModule(file_str, include_dirs=include_dirs)

        for k in kernels:
            setattr(self, k, self.mod.get_function(k))


cf = CudaSourceFile("test_kernel.cu", kernels=["test_kernel"], include_dirs=[notebook_path])
cf.test_kernel(block=(4, 2, 2), grid=(2, 1, 1))

(4, 0, 0): Block (1, 0, 0), Thread (0, 0, 0) -- 42
(5, 0, 0): Block (1, 0, 0), Thread (1, 0, 0) -- 42
(6, 0, 0): Block (1, 0, 0), Thread (2, 0, 0) -- 42
(7, 0, 0): Block (1, 0, 0), Thread (3, 0, 0) -- 42
(4, 1, 0): Block (1, 0, 0), Thread (0, 1, 0) -- 42
(5, 1, 0): Block (1, 0, 0), Thread (1, 1, 0) -- 42
(6, 1, 0): Block (1, 0, 0), Thread (2, 1, 0) -- 42
(7, 1, 0): Block (1, 0, 0), Thread (3, 1, 0) -- 42
(4, 0, 1): Block (1, 0, 0), Thread (0, 0, 1) -- 42
(5, 0, 1): Block (1, 0, 0), Thread (1, 0, 1) -- 42
(6, 0, 1): Block (1, 0, 0), Thread (2, 0, 1) -- 42
(7, 0, 1): Block (1, 0, 0), Thread (3, 0, 1) -- 42
(4, 1, 1): Block (1, 0, 0), Thread (0, 1, 1) -- 42
(5, 1, 1): Block (1, 0, 0), Thread (1, 1, 1) -- 42
(6, 1, 1): Block (1, 0, 0), Thread (2, 1, 1) -- 42
(7, 1, 1): Block (1, 0, 0), Thread (3, 1, 1) -- 42
(0, 0, 0): Block (0, 0, 0), Thread (0, 0, 0) -- 42
(1, 0, 0): Block (0, 0, 0), Thread (1, 0, 0) -- 42
(2, 0, 0): Block (0, 0, 0), Thread (2, 0, 0) -- 42
(3, 0, 0): Block (0, 0, 0), Thr

# Memory Transfers

https://docs.nvidia.com/cuda/cuda-c-best-practices-guide/index.html#memory-optimizations
> Memory optimizations are the most important area for performance.

> it is important to minimize data transfer between the host and the device, even if that means running kernels on the GPU that do not demonstrate any speedup compared with running them on the host CPU.

> Intermediate data structures should be created in device memory, operated on by the device, and destroyed without ever being mapped by the host or copied to host memory.

> higher bandwidth between the host and the device is achieved when using page-locked (or pinned) memory

---

Some useful hints:
- https://medium.com/@rupertt/accelerate-computation-with-pycuda-2c12a6555cc6
  - cuda.memcpy_htod
  - cuda.memcpy_dtoh
- https://wlandau.github.io/gpu/lectures/pycuda/pycuda.pdf
  - In, Out, InOut



In [45]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda import gpuarray
import numpy as np


def to_pair(n):
    return (n >> 8, n & 0xFF)


cf = CudaSourceFile("test_indicies.cu", kernels=["test_indicies"], include_dirs=[notebook_path])

cols = 8
rows = 4
out = cuda.managed_empty(shape=rows * cols, dtype=np.int32, mem_flags=cuda.mem_attach_flags.GLOBAL)
cf.test_indicies(np.int32(cols), out, block=(8, 4, 1), grid=(1, 1, 1))
print("out:", out)
npout = np.array(out)
print("npout", npout)
print("npout[10]", npout[10])
print("npout pair", to_pair(npout[10]))

out: [   0  256  512  768 1024 1280 1536 1792    1  257  513  769 1025 1281
 1537 1793    2  258  514  770 1026 1282 1538 1794    3  259  515  771
 1027 1283 1539 1795]
npout [   0  256  512  768 1024 1280 1536 1792    1  257  513  769 1025 1281
 1537 1793    2  258  514  770 1026 1282 1538 1794    3  259  515  771
 1027 1283 1539 1795]
npout[10] 513
npout pair (2, 1)


(0, 0): out[0] = 0
(1, 0): out[1] = 256
(2, 0): out[2] = 512
(3, 0): out[3] = 768
(4, 0): out[4] = 1024
(5, 0): out[5] = 1280
(6, 0): out[6] = 1536
(7, 0): out[7] = 1792
(0, 1): out[8] = 1
(1, 1): out[9] = 257
(2, 1): out[10] = 513
(3, 1): out[11] = 769
(4, 1): out[12] = 1025
(5, 1): out[13] = 1281
(6, 1): out[14] = 1537
(7, 1): out[15] = 1793
(0, 2): out[16] = 2
(1, 2): out[17] = 258
(2, 2): out[18] = 514
(3, 2): out[19] = 770
(4, 2): out[20] = 1026
(5, 2): out[21] = 1282
(6, 2): out[22] = 1538
(7, 2): out[23] = 1794
(0, 3): out[24] = 3
(1, 3): out[25] = 259
(2, 3): out[26] = 515
(3, 3): out[27] = 771
(4, 3): out[28] = 1027
(5, 3): out[29] = 1283
(6, 3): out[30] = 1539
(7, 3): out[31] = 1795


## managed_empty

https://documen.tician.de/pycuda/driver.html#managed-memory

> CUDA 6.0 adds support for a “Unified Memory” model, which creates a managed virtual memory space that is visible to both CPUs and GPUs. The OS will migrate the physical pages associated with managed memory between the CPU and GPU as needed. This allows a numpy array on the host to be passed to kernels without first creating a DeviceAllocation and manually copying the host data to and from the device.

In [12]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda import gpuarray
import numpy as np

cf = CudaSourceFile("test_pairs.cu", kernels=["test_pairs"], include_dirs=[notebook_path])


out = cuda.managed_empty(shape=(3, 2), dtype=np.int32, mem_flags=cuda.mem_attach_flags.GLOBAL)
cf.test_pairs(out, block=(3, 1, 1), grid=(1, 1, 1))
print("out:", out)
npout = np.array(out)

out: [[ 10 100]
 [ 11 100]
 [ 12 100]]


0: (10, 100)
1: (11, 100)
2: (12, 100)


## malloc

In [18]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda import gpuarray
import numpy as np

cf = CudaSourceFile("test_pairs.cu", kernels=["test_pairs"], include_dirs=[notebook_path])

rows = 3
cols = 2
sizeof_int = 4
out = cuda.mem_alloc(rows * cols * sizeof_int)
cf.test_pairs(out, block=(3, 1, 1), grid=(1, 1, 1))
npout = np.empty((3, 2), dtype=np.int32)
cuda.memcpy_dtoh(npout, out)
print("npout", npout)

npout [[ 10 100]
 [ 11 100]
 [ 12 100]]
0: (10, 100)
1: (11, 100)
2: (12, 100)


## pycuda.Out

In [3]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda import gpuarray
import numpy as np

cf = CudaSourceFile("test_pairs.cu", kernels=["test_pairs"], include_dirs=[notebook_path])

res = np.empty((3, 2), dtype=np.int32)
cf.test_pairs(cuda.Out(res), block=(3, 1, 1), grid=(1, 1, 1))
print("res:", res)

res: [[ 10 100]
 [ 11 100]
 [ 12 100]]
0: (10, 100)
1: (11, 100)
2: (12, 100)


## from_device / device allocated

In [ ]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda import gpuarray
import numpy as np

cf = CudaSourceFile("test_pairs.cu", kernels=["test_pairs"], include_dirs=[notebook_path])

res = np.empty((3, 2), dtype=np.int32)
cf.test_pairs(cuda.Out(res), block=(3, 1, 1), grid=(1, 1, 1))
print("res:", res)

## pycuda.In

In [26]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda import gpuarray
import numpy as np

cf = CudaSourceFile("dump_buf.cu", kernels=["dump_buf"], include_dirs=[notebook_path])

input_buf = np.arange(12).reshape((3, 4)).astype(np.int8)
width = 3
height = 2
cf.dump_buf(np.int32(width), np.int32(height), cuda.In(input_buf), block=(3, 3, 1))
cuda.context.synchronize()
print("input_buf", input_buf)

input_buf [[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]
(0, 0): val 0
(1, 0): val 1
(2, 0): val 2
(0, 1): val 3
(1, 1): val 4
(2, 1): val 5
(0, 2): val 6
(1, 2): val 7
(2, 2): val 8


## TODO
- [ ] from_device
  - https://documen.tician.de/pycuda/driver.html#pycuda.driver.from_device
- [ ] mem_alloc_pitch
  - https://documen.tician.de/pycuda/driver.html#pycuda.driver.mem_alloc_pitch
- [ ] page-locked host memory
  - https://documen.tician.de/pycuda/driver.html#pagelocked-host-memory
  - https://docs.nvidia.com/cuda/cuda-c-best-practices-guide/index.html#pinned-memory
- [ ] memory pools
  - https://documen.tician.de/pycuda/util.html#device-based-memory-pool
- [ ] structs
  - https://github.com/minrk/PyCUDA/blob/master/doc/source/tutorial.rst#structures
- [ ] Shared memory
  - https://developer.nvidia.com/blog/using-shared-memory-cuda-cc/

# Events

## Daisy Chain

In [12]:
import os
import pycuda.autoinit
from pycuda.compiler import SourceModule

notebook_path = os.path.abspath("")


class CudaSourceFile:
    def __init__(
        self, filename: str, kernels: list[str] = list(), include_dirs: list[str] = list()
    ) -> None:
        with open(filename) as f:
            file_str = f.read()
            self.mod = SourceModule(
                file_str,
                include_dirs=include_dirs,
                # options=["-rdc=true"],
            )

        for k in kernels:
            setattr(self, k, self.mod.get_function(k))


import pycuda.driver as cuda
import pycuda.autoinit
from pycuda import gpuarray
import numpy as np

cf = CudaSourceFile("daisy_chain.cu", kernels=["k1", "k2"], include_dirs=[notebook_path])
e = cuda.Event()
cf.k1(e, block=(2, 1, 1))
cf.k2(e, block=(2, 1, 1))

# grid = np.array(
#     [
#         [3, 3, 0, 5],
#         [0, 1, 0, 5],
#         [0, 0, 4, 2],
#     ],
#     dtype=np.int16,
# )
# print(f"grid:\n{grid}")

# res = np.zeros_like(grid)

# width = grid.shape[0]
# height = grid.shape[1]
# np_width = np.int32(width)
# np_height = np.int32(height)

# cf.fe_single(cuda.Out(res), np_width, np_height, cuda.In(grid), block=(width, height, 1))
# print(f"res:\n{res}")

CompileError: nvcc compilation of /tmp/tmp_kjz3mqj/kernel.cu failed
[command: nvcc --cubin -arch sm_89 -I/home/apowers/Projects/roc/experiments/2024.08.30-07.27.46-pycuda-learning -I/home/apowers/Projects/roc/.venv/lib/python3.11/site-packages/pycuda/cuda kernel.cu]
[stderr:
ptxas fatal   : Unresolved extern function '__cudaCDP2EventCreateWithFlags'
]

# CUDA Python

NVIDIA's version of PyCUDA supports graphs for streaming

## Basic

Copied from https://nvidia.github.io/cuda-python/overview.html

In [4]:
from cuda import cuda, nvrtc
import numpy as np
import pathlib


def _cudaGetErrorEnum(error):
    if isinstance(error, cuda.CUresult):
        err, name = cuda.cuGetErrorName(error)
        return name if err == cuda.CUresult.CUDA_SUCCESS else "<unknown>"
    elif isinstance(error, nvrtc.nvrtcResult):
        return nvrtc.nvrtcGetErrorString(error)[1]
    else:
        raise RuntimeError("Unknown error type: {}".format(error))


def checkCudaErrors(result):
    # if result[0].value and _cudaGetErrorEnum(result[0]) == NVRTC_ERROR_COMPILATION
    if result[0].value:
        print("ERR STRING", result)
        raise RuntimeError(
            "CUDA error code={}({})".format(result[0].value, _cudaGetErrorEnum(result[0]))
        )
    if len(result) == 1:
        return None
    elif len(result) == 2:
        return result[1]
    else:
        return result[1:]


with open("saxpy.cu") as f:
    saxpy = f.read()

# Initialize CUDA Driver API
checkCudaErrors(cuda.cuInit(0))

# Retrieve handle for device 0
cuDevice = checkCudaErrors(cuda.cuDeviceGet(0))

# Derive target architecture for device 0
major = checkCudaErrors(
    cuda.cuDeviceGetAttribute(
        cuda.CUdevice_attribute.CU_DEVICE_ATTRIBUTE_COMPUTE_CAPABILITY_MAJOR, cuDevice
    )
)
minor = checkCudaErrors(
    cuda.cuDeviceGetAttribute(
        cuda.CUdevice_attribute.CU_DEVICE_ATTRIBUTE_COMPUTE_CAPABILITY_MINOR, cuDevice
    )
)
arch_arg = bytes(f"--gpu-architecture=compute_{major}{minor}", "ascii")

# Create program
prog = checkCudaErrors(nvrtc.nvrtcCreateProgram(str.encode(saxpy), b"saxpy.cu", 0, [], []))

# Compile program
opts = [b"--fmad=false", arch_arg]
ret = nvrtc.nvrtcCompileProgram(prog, len(opts), opts)
# ret = nvrtc.nvrtcCompileProgram(prog, 0, [])
log_sz = checkCudaErrors(nvrtc.nvrtcGetProgramLogSize(prog))
print("Log size is", log_sz)
buf = b" " * log_sz
checkCudaErrors(nvrtc.nvrtcGetProgramLog(prog, buf))
print("Log:", buf.decode())
checkCudaErrors(ret)

# Get PTX from compilation
ptxSize = checkCudaErrors(nvrtc.nvrtcGetPTXSize(prog))
ptx = b" " * ptxSize
checkCudaErrors(nvrtc.nvrtcGetPTX(prog, ptx))

# Create context
context = checkCudaErrors(cuda.cuCtxCreate(0, cuDevice))

# Load PTX as module data and retrieve function
ptx = np.char.array(ptx)
# Note: Incompatible --gpu-architecture would be detected here
module = checkCudaErrors(cuda.cuModuleLoadData(ptx.ctypes.data))
kernel = checkCudaErrors(cuda.cuModuleGetFunction(module, b"saxpy"))

NUM_THREADS = 512  # Threads per block
NUM_BLOCKS = 32768  # Blocks per grid

a = np.array([2.0], dtype=np.float32)
n = np.array(NUM_THREADS * NUM_BLOCKS, dtype=np.uint32)
bufferSize = n * a.itemsize

hX = np.random.rand(n).astype(dtype=np.float32)
hY = np.random.rand(n).astype(dtype=np.float32)
hOut = np.zeros(n).astype(dtype=np.float32)

dXclass = checkCudaErrors(cuda.cuMemAlloc(bufferSize))
dYclass = checkCudaErrors(cuda.cuMemAlloc(bufferSize))
dOutclass = checkCudaErrors(cuda.cuMemAlloc(bufferSize))

stream = checkCudaErrors(cuda.cuStreamCreate(0))

checkCudaErrors(cuda.cuMemcpyHtoDAsync(dXclass, hX.ctypes.data, bufferSize, stream))
checkCudaErrors(cuda.cuMemcpyHtoDAsync(dYclass, hY.ctypes.data, bufferSize, stream))

# The following code example is not intuitive
# Subject to change in a future release
dX = np.array([int(dXclass)], dtype=np.uint64)
dY = np.array([int(dYclass)], dtype=np.uint64)
dOut = np.array([int(dOutclass)], dtype=np.uint64)

args = [a, dX, dY, dOut, n]
args = np.array([arg.ctypes.data for arg in args], dtype=np.uint64)

checkCudaErrors(
    cuda.cuLaunchKernel(
        kernel,
        NUM_BLOCKS,  # grid x dim
        1,  # grid y dim
        1,  # grid z dim
        NUM_THREADS,  # block x dim
        1,  # block y dim
        1,  # block z dim
        0,  # dynamic shared memory
        stream,  # stream
        args.ctypes.data,  # kernel arguments
        0,  # extra (ignore)
    )
)

checkCudaErrors(cuda.cuMemcpyDtoHAsync(hOut.ctypes.data, dOutclass, bufferSize, stream))
checkCudaErrors(cuda.cuStreamSynchronize(stream))

# Assert values are same after running kernel
hZ = a * hX + hY
if not np.allclose(hOut, hZ):
    raise ValueError("Error outside tolerance for host-device vectors")

checkCudaErrors(cuda.cuStreamDestroy(stream))
checkCudaErrors(cuda.cuMemFree(dXclass))
checkCudaErrors(cuda.cuMemFree(dYclass))
checkCudaErrors(cuda.cuMemFree(dOutclass))
checkCudaErrors(cuda.cuModuleUnload(module))
checkCudaErrors(cuda.cuCtxDestroy(context))

Log size is 159
Log: saxpy.cu(5): warning #177-D: variable "xxx" was declared but never referenced

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

 


## Test Kernel

In [18]:
from cuda import cuda, nvrtc
import numpy as np
import pathlib


def _cudaGetErrorEnum(error):
    if isinstance(error, cuda.CUresult):
        err, name = cuda.cuGetErrorName(error)
        return name if err == cuda.CUresult.CUDA_SUCCESS else "<unknown>"
    elif isinstance(error, nvrtc.nvrtcResult):
        return nvrtc.nvrtcGetErrorString(error)[1]
    else:
        raise RuntimeError("Unknown error type: {}".format(error))


def checkCudaErrors(result):
    # if result[0].value and _cudaGetErrorEnum(result[0]) == NVRTC_ERROR_COMPILATION
    if result[0].value:
        print("ERR STRING", result)
        raise RuntimeError(
            "CUDA error code={}({})".format(result[0].value, _cudaGetErrorEnum(result[0]))
        )
    if len(result) == 1:
        return None
    elif len(result) == 2:
        return result[1]
    else:
        return result[1:]


with open("test_kernel.cu") as f:
    code = 'extern "C" {\n' + f.read() + "\n}\n"
print(f"CODE:\n-------\n{code}\n-------\n")

# def _find_cuda_incl_path() -> pathlib.Path:
#     "Tries to find the CUDA include path."
#     cuda_path = os.getenv("CUDA_PATH")
#     if cuda_path is None:
#         if sys.platform == "linux":
#             cuda_path = pathlib.Path("/usr/local/cuda/include")
#             if not (cuda_path.exists() and cuda_path.is_dir()):
#                 cuda_path = None
#         elif sys.platform == "win32":
#             ...
#         elif sys.platform == "darwin":
#             ...
#     else:
#         cuda_path = pathlib.Path(cuda_path)
#         cuda_path /= "include"

#     return cuda_path


# CUDA_HOME = os.getenv("CUDA_HOME")
# if CUDA_HOME == None:
#     CUDA_HOME = os.getenv("CUDA_PATH")
# if CUDA_HOME == None:
#     raise RuntimeError("Environment variable CUDA_HOME or CUDA_PATH is not set")
# include_dirs = os.path.join(CUDA_HOME, "include")

# Initialize CUDA Driver API
checkCudaErrors(cuda.cuInit(0))

# Retrieve handle for device 0
cuDevice = checkCudaErrors(cuda.cuDeviceGet(0))

# Derive target architecture for device 0
major = checkCudaErrors(
    cuda.cuDeviceGetAttribute(
        cuda.CUdevice_attribute.CU_DEVICE_ATTRIBUTE_COMPUTE_CAPABILITY_MAJOR, cuDevice
    )
)
minor = checkCudaErrors(
    cuda.cuDeviceGetAttribute(
        cuda.CUdevice_attribute.CU_DEVICE_ATTRIBUTE_COMPUTE_CAPABILITY_MINOR, cuDevice
    )
)
arch_arg = bytes(f"--gpu-architecture=compute_{major}{minor}", "ascii")

# Create program
prog = checkCudaErrors(nvrtc.nvrtcCreateProgram(str.encode(code), b"test_kernel.cu", 0, [], []))

# Compile program
opts = [b"--fmad=false", arch_arg]
# ret = nvrtc.nvrtcCompileProgram(prog, len(opts), opts)
ret = nvrtc.nvrtcCompileProgram(prog, 0, [])
log_sz = checkCudaErrors(nvrtc.nvrtcGetProgramLogSize(prog))
print("Log size is", log_sz)
buf = b" " * log_sz
checkCudaErrors(nvrtc.nvrtcGetProgramLog(prog, buf))
print("Log:", buf.decode())
checkCudaErrors(ret)

# Get PTX from compilation
ptxSize = checkCudaErrors(nvrtc.nvrtcGetPTXSize(prog))
ptx = b" " * ptxSize
checkCudaErrors(nvrtc.nvrtcGetPTX(prog, ptx))

# Create context
context = checkCudaErrors(cuda.cuCtxCreate(0, cuDevice))

# Load PTX as module data and retrieve function
ptx = np.char.array(ptx)
module = checkCudaErrors(cuda.cuModuleLoadData(ptx.ctypes.data))
kernel = checkCudaErrors(cuda.cuModuleGetFunction(module, b"test_kernel"))
stream = checkCudaErrors(cuda.cuStreamCreate(0))

# args = np.array([], np.uint64)

checkCudaErrors(
    cuda.cuLaunchKernel(
        kernel,
        3,  # grid x dim
        1,  # grid y dim
        1,  # grid z dim
        2,  # block x dim
        1,  # block y dim
        1,  # block z dim
        0,  # dynamic shared memory
        stream,  # stream
        #    args.ctypes.data,  # kernel arguments
        0,  # kernel arguments
        0,  # extra (ignore)
    )
)

checkCudaErrors(cuda.cuStreamSynchronize(stream))
checkCudaErrors(cuda.cuStreamDestroy(stream))
checkCudaErrors(cuda.cuModuleUnload(module))
checkCudaErrors(cuda.cuCtxDestroy(context))

CODE:
-------
extern "C" {
#include "test.h"

__global__ void test_kernel() {
  printf("(%d, %d, %d): Block (%d, %d, %d), Thread (%d, %d, %d) -- %d\n", MY_X,
         MY_Y, MY_Z, blockIdx.x, blockIdx.y, blockIdx.z, threadIdx.x,
         threadIdx.y, threadIdx.z, MY_THING);
}
}

-------

Log size is 1
Log:  
(2, 0, 0): Block (1, 0, 0), Thread (0, 0, 0) -- 42
(3, 0, 0): Block (1, 0, 0), Thread (1, 0, 0) -- 42
(0, 0, 0): Block (0, 0, 0), Thread (0, 0, 0) -- 42
(1, 0, 0): Block (0, 0, 0), Thread (1, 0, 0) -- 42
(4, 0, 0): Block (2, 0, 0), Thread (0, 0, 0) -- 42
(5, 0, 0): Block (2, 0, 0), Thread (1, 0, 0) -- 42


# Feature Extractors

## Single

In [4]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda import gpuarray
import numpy as np

cf = CudaSourceFile("fe_single.cu", kernels=["fe_single"], include_dirs=[notebook_path])

grid = np.array(
    [
        [3, 3, 0, 5],
        [0, 1, 0, 5],
        [0, 0, 4, 2],
    ],
    dtype=np.int16,
)
print(f"grid:\n{grid}")

res = np.zeros_like(grid)

width = grid.shape[0]
height = grid.shape[1]
np_width = np.int32(width)
np_height = np.int32(height)

cf.fe_single(cuda.Out(res), np_width, np_height, cuda.In(grid), block=(width, height, 1))
print(f"res:\n{res}")

grid:
[[3 3 0 5]
 [0 1 0 5]
 [0 0 4 2]]
(0, 0) val: 3, unique 0
(1, 0) val: 3, unique 0
(2, 0) val: 0, unique 0
(0, 1) val: 5, unique 0
(1, 1) val: 0, unique 0
(2, 1) val: 1, unique 1
(0, 2) val: 0, unique 0
(1, 2) val: 5, unique 0
(2, 2) val: 0, unique 0
(0, 3) val: 0, unique 0
(1, 3) val: 4, unique 1
(2, 3) val: 2, unique 1
res:
[[0 0 0 0]
 [0 1 0 0]
 [0 0 1 1]]


# Profiler

https://documen.tician.de/pycuda/driver.html#profiler-control

`driver.initialize_profiler(config_file, output_file, output_mode)`

`driver.start_profiler()`

`driver.stop_profiler()`